In [40]:
import pandas as pd
data = pd.read_csv('./dscan_result/data/sim_hash_result.csv')
print(data.head())

dict ={}
        
for i in range(len(data['label'])):
    if data['label'][i] not in dict:
        dict[data['label'][i]] = len(dict)
print(dict)

data.label.value_counts()

                                             feature  label
0  400 The plain HTTP request was sent to HTTPS p...  nginx
1  HTTP/1.1 400 Bad Request Content-Type: text/ht...    iis
2  HTTP/1.1 400 Bad Request Server: nginx/1.21.4 ...  nginx
3  HTTP/1.1 400 Bad Request Server: nginx/1.23.1 ...  nginx
4  401 Authorization Required 401 Authorization R...  nginx
{'nginx': 0, 'iis': 1, 'lighttpd': 2, 'boa': 3, 'tomcat': 4, 'rompager': 5, 'http server': 6, 'micro httpd': 7, 'jetty': 8, 'tengine': 9}


label
http server    14438
nginx          12569
iis             3582
jetty            951
lighttpd         630
tomcat           331
tengine          260
boa              248
micro httpd      232
rompager         121
Name: count, dtype: int64

In [41]:
# 生成class配置文件
with open('./dscan_result/data/class.txt', 'a+') as f:
    for i in dict.keys():
        f.write(i + '\n')
f.close()

In [42]:
# 生成label配置文件
import pickle

result_single = {}
result_multi = {}

a = []
b = []

for item in data['label']:
    a.append(item)

for i in set(a):
    result_single[i] = a.count(i)

label_list = sorted([_ for _ in result_single.items()], key=lambda x: x[1], reverse=True)
print(label_list)

f = open('./dscan_result/data/label.pkl', 'wb')
pickle.dump(label_list, f)
f.close()


[('http server', 14438), ('nginx', 12569), ('iis', 3582), ('jetty', 951), ('lighttpd', 630), ('tomcat', 331), ('tengine', 260), ('boa', 248), ('micro httpd', 232), ('rompager', 121)]


In [43]:

for i in range(len(data['label'])):
    data['label'][i] = dict[data['label'][i]]

for i in range(len(data['feature'])):
    data['feature'][i] = str(data['feature'][i]).lstrip().rstrip()
print(data.head(10))

                                             feature label
0  400 The plain HTTP request was sent to HTTPS p...     0
1  HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
2  HTTP/1.1 400 Bad Request Server: nginx/1.21.4 ...     0
3  HTTP/1.1 400 Bad Request Server: nginx/1.23.1 ...     0
4  401 Authorization Required 401 Authorization R...     0
5  400 The plain HTTP request was sent to HTTPS p...     0
6  HTTP/1.0 400 Bad Request Content-Type: text/ht...     2
7  HTTP/1.1 400 Bad Request Content-Type: text/ht...     0
8  400 The plain HTTP request was sent to HTTPS p...     0
9  404 Not Found 404 Not Found nginx/1.20.2 HTTP/...     0


In [44]:
def simpleSampling(group, frac):
    return group.sample(frac=frac)

train_df = data.groupby(data.label).apply(simpleSampling, 0.5).sample(frac=1, ignore_index = True)
print(train_df)
train_df.label.value_counts()

                                                 feature label
0      404 page not found HTTP/1.1 505 HTTP Version N...     0
1      HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
2      { `` error '' : { `` statusCode '' :404   '' n...     0
3      HTTP/1.1 400 Bad Request Data: DATE Server: Ap...     6
4      HTTP/1.1 505 Unknown Version Content-Type: tex...     8
...                                                  ...   ...
16676  { `` detail '' : `` not found . '' } HTTP/1.1 ...     0
16677  HTTP/1.1 503 Service Unavailable Content-Type:...     6
16678  HTTP/1.0 200 OK Data: DATE Server: Apache/2.4....     6
16679  HTTP/1.1 400 Bad Request Data: DATE Server: Ap...     6
16680  HTTP/1.1 400 BAD_REQUEST Content-Length: CONTE...     6

[16681 rows x 2 columns]


label
6    7219
0    6284
1    1791
8     476
2     315
4     166
9     130
3     124
7     116
5      60
Name: count, dtype: int64

In [45]:
data = pd.concat([data, train_df.sample(frac=1, ignore_index=True)]).drop_duplicates(keep=False)
# data = data.append(train_df.sample(frac=1, ignore_index=True)).drop_duplicates(keep = False)
print(data)

                                                 feature label
0      400 The plain HTTP request was sent to HTTPS p...     0
1      HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
3      HTTP/1.1 400 Bad Request Server: nginx/1.23.1 ...     0
4      401 Authorization Required 401 Authorization R...     0
6      HTTP/1.0 400 Bad Request Content-Type: text/ht...     2
...                                                  ...   ...
33356  HTTP/1.0 400 Bad Request Data: DATE Server: Bo...     3
33357  HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
33358  403 - Forbidden : Access is denied . Server Er...     1
33360  HTTP/1.0 400 Bad Request Content-Type: text/ht...     2
33361  HTTP/1.0 400 Bad Request Content-Type: text/ht...     2

[16644 rows x 2 columns]


In [46]:
val_df = data.groupby(data.label).apply(simpleSampling, 0.5).sample(frac=1, ignore_index = True)
print(val_df.label.value_counts())

# test_df = data.append(val_df).drop_duplicates(keep=False)
test_df = pd.concat([data, val_df]).drop_duplicates(keep=False)

print(test_df.label.value_counts())
train_df.to_csv('./dscan_result/data/train.txt', header=None, index=None)
val_df.to_csv('./dscan_result/data/dev.txt', header=None, index=None)
test_df.to_csv('./dscan_result/data/test.txt', header=None, index=None)

label
6    3610
0    3126
1     895
8     238
2     156
4      82
9      65
3      62
7      58
5      30
Name: count, dtype: int64
label
6    3609
0    3126
1     895
8     237
2     156
4      83
9      65
3      62
7      58
5      31
Name: count, dtype: int64
